In [30]:
%run Solution.ipynb

In [31]:
def calculate_delta_r9(T):
    return G0('AlCl3', T) + G0('NH3', T) - G0('AlN', T) - 3 * G0('HCl', T)

def calculate_delta_r10(T):
    return G0('GaCl', T) + G0('NH3', T) - G0('GaN', T) - G0('HCl', T) - G0('H2', T)

In [32]:
Pa = 100000

def K9(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r9(T) / (R * T)) / Pa

def K10(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r10(T) / (R * T))

In [33]:
T = 1100.0
print(K9(T))
print(K10(T))

3.100281771447924e-08
0.13120264119198397


In [34]:
print(D('NH3', T))
print(D('GaCl', T))
print(D('AlCl3', T))
print(D('HCl', T))
print(D('H2', T))

0.0007886217210652681
0.0004891294645967424
0.00039166855641119605
0.0007030166473273947
0.0022512831014624562


In [35]:

def run_1(xg):
    PG = {
       'NH3':1500,
       'GaCl':(30 * (1 - xg)),
       'AlCl3':(30 * xg),
       'H2':0,
       'HCl':0,
       'N2':98470
    }


    x, AlCl3, GaCl, NH3, HCl, H2 = sy.symbols('x AlCl3 GaCl NH3 HCl H2')
    Xmat = sy.Matrix([
        AlCl3 * NH3 - K9(T) * x * (HCl)**3,
        GaCl * NH3 - K10(T) * (1 - x) * HCl * H2,
        D('HCl', T) * (PG['HCl'] - HCl) + 2 * D('H2', T) * (PG['H2'] - H2) + 3 * D('NH3', T) * (PG['NH3'] - NH3),
        3 * D('AlCl3', T) * (PG['AlCl3'] - AlCl3) + D('GaCl', T) * (PG['GaCl'] - GaCl) + D('HCl', T) * (PG['HCl'] - HCl),
        D('AlCl3', T) * (PG['AlCl3'] - AlCl3) + D('GaCl', T) * (PG['GaCl'] - GaCl) - D('NH3', T) * (PG['NH3'] - NH3),
        (1 - x) * D('AlCl3', T) * (PG['AlCl3'] - AlCl3) - x * D('GaCl', T) * (PG['GaCl'] - GaCl),
        
    ])
    
    Y = [x, AlCl3, GaCl, NH3, HCl, H2]
    ans = newton_method(Xmat * 10**40, Y, [1, 1, 1, 1, 1, 1])
    print(ans)

for d in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]:
    run_1(d)

11
[7.25932464025329e-17, 1.78141654234548e-19, 0.0120397060547704, 1481.40046796693, 20.8643636230920, 6.51539335652517]
12
[0.166855689777968, 6.65776163219690e-8, 0.0102711142324109, 1482.14087415813, 26.7193063696268, 5.21217577571369]
13
[0.348130210470462, 2.51589296382878e-7, 0.00734465474480411, 1482.88056226012, 32.5750545139192, 3.90920976130515]
12
[0.545768813613789, 6.47343607624247e-7, 0.00402354365508033, 1483.62000569071, 38.4310768879955, 2.60632949165522]
13
[0.762110515021665, 1.38261470478399e-6, 0.00121369481382654, 1484.35976639172, 44.2867429798042, 1.30333814160438]
13
[1.00000000000000, 2.63167361243703e-6, -8.45807379229063e-23, 1485.10051733584, 50.1412976759286, -3.54080502721730e-13]


In [36]:

def run_2(xg):
    PG = {
       'NH3':1500,
       'GaCl':(30 * (1 - xg)),
       'AlCl3':(30 * xg),
       'H2':9847,
       'HCl':0,
       'N2':98470 - 9847
    }


    x, AlCl3, GaCl, NH3, HCl, H2 = sy.symbols('x AlCl3 GaCl NH3 HCl H2')
    Xmat = sy.Matrix([
        AlCl3 * NH3 - K9(T) * x * (HCl)**3,
        GaCl * NH3 - K10(T) * (1 - x) * HCl * H2,
        D('HCl', T) * (PG['HCl'] - HCl) + 2 * D('H2', T) * (PG['H2'] - H2) + 3 * D('NH3', T) * (PG['NH3'] - NH3),
        3 * D('AlCl3', T) * (PG['AlCl3'] - AlCl3) + D('GaCl', T) * (PG['GaCl'] - GaCl) + D('HCl', T) * (PG['HCl'] - HCl),
        D('AlCl3', T) * (PG['AlCl3'] - AlCl3) + D('GaCl', T) * (PG['GaCl'] - GaCl) - D('NH3', T) * (PG['NH3'] - NH3),
        (1 - x) * D('AlCl3', T) * (PG['AlCl3'] - AlCl3) - x * D('GaCl', T) * (PG['GaCl'] - GaCl),
        
    ])
    
    Y = [x, AlCl3, GaCl, NH3, HCl, H2]
    ans = newton_method(Xmat * 10**40, Y, [1, 1, 1, 1, 1, 1])
    print(ans)

for d in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]:
    run_2(d)

93
[4.15970486855212e-15, 2.43167289099778e-18, 11.2988182463579, 1488.40090404036, 13.0114970296363, 9851.06314914924]
109
[0.279898959552174, 3.76633378510502e-8, 11.6394360954248, 1489.35366969460, 18.6282217152204, 9849.68554230289]
101
[0.547287674844102, 1.90868357978791e-7, 10.0515440440016, 1489.11030931697, 25.5867224212156, 9848.72693696466]
68
[0.751036806694370, 5.83478069823155e-7, 7.22205333257345, 1488.09686820502, 33.4090749418425, 9848.03809000874]
29
[0.895941511644018, 1.35146333841027e-6, 3.76794982514023, 1486.69602161433, 41.6660061919844, 9847.48495078489]
25
[0.999999999999988, 2.63167361252312e-6, 8.89974851839459e-13, 1485.10051733428, 50.1412976758788, 9846.99999998964]
